In [ ]:
import matplotlib.pyplot as plt
from fpdf import FPDF


X = [
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
]
y = [0, 0, 0, 1]


w1 = 0.0
w2 = 0.0
b = 0.0
learning_rate = 0.5
max_epochs = 20


history_error = []
history_w1 = []
history_w2 = []
history_b = []
history_details = []


def activation(z):
    return 1 if z >= 0 else 0


for epoch in range(1, max_epochs + 1):
    total_error = 0
    details = f"Época {epoch}:\n"
    
    for i in range(len(X)):
        x1 = X[i][0]
        x2 = X[i][1]
        yi = y[i]

        
        z = (x1 * w1) + (x2 * w2) + b
        y_pred = activation(z)
        error = yi - y_pred
        total_error += abs(error)

        
        details += f"\nAmostra {i+1}:\n"
        details += f"  Entrada: ({x1}, {x2})\n"
        details += f"  Cálculo: z = ({x1}*{w1:.2f}) + ({x2}*{w2:.2f}) + ({b:.2f}) = {z:.2f}\n"
        details += f"  Aplicação da ativação: y_pred = {y_pred}\n"
        details += f"  Saída esperada: {yi}, Erro: {error}\n"
        
        
        old_w1, old_w2, old_b = w1, w2, b
        w1 += learning_rate * error * x1
        w2 += learning_rate * error * x2
        b += learning_rate * error

        details += f"  Atualização:\n"
        details += f"    w1 = {old_w1:.2f} + ({learning_rate} * {error} * {x1}) = {w1:.2f}\n"
        details += f"    w2 = {old_w2:.2f} + ({learning_rate} * {error} * {x2}) = {w2:.2f}\n"
        details += f"    b = {old_b:.2f} + ({learning_rate} * {error}) = {b:.2f}\n"

    
    history_error.append(total_error)
    history_w1.append(w1)
    history_w2.append(w2)
    history_b.append(b)
    history_details.append((epoch, total_error, details))
    
    if total_error == 0:
        break


plt.figure(figsize=(8, 4))
plt.plot(range(1, len(history_error)+1), history_error, marker='o')
plt.title('Erro total por Época')
plt.xlabel('Época')
plt.ylabel('Erro Total')
plt.grid(True)
plt.tight_layout()
plt.savefig('erro_por_epoca.png')
plt.close()

plt.figure(figsize=(8, 4))
plt.plot(range(1, len(history_w1)+1), history_w1, label='Peso w1', marker='o')
plt.plot(range(1, len(history_w2)+1), history_w2, label='Peso w2', marker='o')
plt.plot(range(1, len(history_b)+1), history_b, label='Bias', marker='x')
plt.title('Evolução dos Pesos e Bias por Época')
plt.xlabel('Época')
plt.ylabel('Valor')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('pesos_bias_evolucao.png')
plt.close()

plt.figure(figsize=(8, 6))

for i in range(len(X)):
    if y[i] == 0:
        plt.scatter(X[i][0], X[i][1], color='red', label='Classe 0' if i == 0 else "")
    else:
        plt.scatter(X[i][0], X[i][1], color='blue', label='Classe 1' if i == 3 else "")

x_vals = [-0.5, 1.5]
if w2 != 0:
    y_vals = [-(w1 * x + b) / w2 for x in x_vals]
    plt.plot(x_vals, y_vals, 'k--', label='Fronteira de decisão')
    slope = -w1 / w2
    intercept = -b / w2
    equation_text = f"Equação da reta: y = {slope:.2f}x + {intercept:.2f}"
else:
    x_const = -b / w1
    plt.axvline(x=x_const, color='k', linestyle='--', label='Fronteira de decisão')
    equation_text = f"Equação da reta: x = {x_const:.2f}"

plt.title('Separação das Classes pelo Perceptron\n' + equation_text, fontsize=12)
plt.xlabel('Entrada x1')
plt.ylabel('Entrada x2')
plt.legend()
plt.xlim(-0.5, 1.5)
plt.ylim(-0.5, 1.5)
plt.grid(True)
plt.tight_layout()
plt.savefig('bounding_line.png')
plt.close()


class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 14)
        self.cell(0, 10, 'Relatório de Treinamento - Perceptron AND', 0, 1, 'C')
        self.ln(5)
        
        self.set_font('Arial', '', 11)
        self.cell(0, 10, 'Integrantes do Grupo:', 0, 1, 'L')
        self.cell(0, 8, '- João Pedro Coelho Barbosa', 0, 1, 'L')
        self.cell(0, 8, '- Luis Eduardo Alencar Melo', 0, 1, 'L')
        self.cell(0, 8, '- Fabiana Lustosa da Cunha', 0, 1, 'L')
        self.ln(5)

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(3)

    def chapter_body(self, body):
        self.set_font('Arial', '', 11)
        self.multi_cell(0, 8, body)
        self.ln()

pdf = PDF()
pdf.add_page()

pdf.chapter_title('Resumo Final:')
resumo = f"""\
Pesos finais:
  - w1 = {w1:.4f}
  - w2 = {w2:.4f}
Bias final:
  - b = {b:.4f}

Número de épocas até convergência: {len(history_error)}
Taxa de aprendizado: {learning_rate}
"""
pdf.chapter_body(resumo)

pdf.chapter_title('Cálculo Detalhado Época por Época:')
for (epoca, erro, detalhes) in history_details:
    texto_epoca = f"Época {epoca} - Erro Total: {erro}\n{detalhes}\n"
    pdf.chapter_body(texto_epoca)

pdf.chapter_title('Gráficos:')
pdf.image('erro_por_epoca.png', x=10, w=190)
pdf.ln(5)
pdf.image('pesos_bias_evolucao.png', x=10, w=190)
pdf.ln(5)
pdf.image('bounding_line.png', x=10, w=190)

pdf.output('relatorio_perceptron_AND_final_completo.pdf')

print("Relatório PDF gerado com sucesso: 'relatorio_perceptron_AND_final_completo.pdf'")


Relatório PDF gerado com sucesso: 'relatorio_perceptron_AND_final_completo.pdf'
